In [6]:
import os
import random
import shutil
import time
import warnings
#os.environ["CUDA_VISIBLE_DEVICES"] = "5"
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [7]:
# Data loading code

##!!!! change the valdir to your data path
valdir = '/data/public/imagenet2012/val'
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

batch_size = 256
val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(valdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ])),
    batch_size=batch_size, shuffle=False,
    num_workers=4, pin_memory=True)

In [12]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
           
            images = images.cuda()
            target = target.cuda()

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % 10 == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'
    
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].contiguous().view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

##  Arnold and iArnold

In [13]:
import copy

def arnold(img, key):
    N, a, b, c, d = key  #  key=[1,1,1,1,2]
    h, w = img.shape[: 2]
    new_img = copy.deepcopy(img)

    for i in range(N):
        
        for x in range(h):
            
            for y in range(w):
                
                nx = ((a * x + b * y) % w + w) % w
                ny = ((c * x + d * y) % w + w) % w
                nx = int(nx)
                
                ny = int(ny)
                
                new_img[nx, ny] = img[x, y]
        img = copy.deepcopy(new_img)
    return new_img


def iarnold(img, key):
    N, a, b, c, d = key
    #matrix = np.mat([[a, b], [c, d]]).I
    [[a, b], [c, d]] = [[(a*b+1),-a],[-b,1]]
    return arnold(img, [N, a, b, c, d])

## Load a pre-trained model 

In [14]:
import torchvision.models as models



vgg16 = models.vgg16(pretrained=True)

# resnet18 = models.resnet18(pretrained=True)
#densenet = models.densenet121(pretrained=True)
#inception = models.inception_v3(pretrained=True)
# googlenet = models.googlenet(pretrained=True)
#shufflenet = models.shufflenet_v2_x1_0(pretrained=True)
#mobilenet = models.mobilenet_v2(pretrained=True)

In [15]:
import torchvision.models as models
cudnn.benchmark = True
criterion = nn.CrossEntropyLoss().cuda()

# change this model
# model = models.googlenet(pretrained=True)
#model = models.resnet101(pretrained=True)
#model = models.mobilenet_v2(pretrained=True)
#model = models.densenet121(pretrained=True)

model = models.vgg16(pretrained=True)
model = model.cuda()

# 每层名字
for name, param in model.named_parameters():
    print(name,'    ', param.size())



## test the original accuracy
validate(val_loader, model, criterion) 

features.0.weight      torch.Size([64, 3, 3, 3])
features.0.bias      torch.Size([64])
features.2.weight      torch.Size([64, 64, 3, 3])
features.2.bias      torch.Size([64])
features.5.weight      torch.Size([128, 64, 3, 3])
features.5.bias      torch.Size([128])
features.7.weight      torch.Size([128, 128, 3, 3])
features.7.bias      torch.Size([128])
features.10.weight      torch.Size([256, 128, 3, 3])
features.10.bias      torch.Size([256])
features.12.weight      torch.Size([256, 256, 3, 3])
features.12.bias      torch.Size([256])
features.14.weight      torch.Size([256, 256, 3, 3])
features.14.bias      torch.Size([256])
features.17.weight      torch.Size([512, 256, 3, 3])
features.17.bias      torch.Size([512])
features.19.weight      torch.Size([512, 512, 3, 3])
features.19.bias      torch.Size([512])
features.21.weight      torch.Size([512, 512, 3, 3])
features.21.bias      torch.Size([512])
features.24.weight      torch.Size([512, 512, 3, 3])
features.24.bias      torch.Size(

tensor(71.5940, device='cuda:0')

## Encryption

In [16]:
import time 
import numpy as np

# layer 1
weight = model.features[0].weight
weight_half = weight[:3,:3]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec in python'.format(total))

model.features[0].weight.data[:3,:3] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion)  

layer half weights size is:  torch.Size([3, 3, 3, 3])
need 0.000163 sec in python
Test: [  0/196]	Time  2.962 ( 2.962)	Loss 5.8738e-01 (5.8738e-01)	Acc@1  85.16 ( 85.16)	Acc@5  96.88 ( 96.88)
Test: [ 10/196]	Time  0.267 ( 0.826)	Loss 1.1062e+00 (8.5168e-01)	Acc@1  71.48 ( 78.30)	Acc@5  92.97 ( 94.00)
Test: [ 20/196]	Time  2.391 ( 0.892)	Loss 9.2269e-01 (8.6629e-01)	Acc@1  80.86 ( 78.12)	Acc@5  92.58 ( 93.84)
Test: [ 30/196]	Time  0.267 ( 0.865)	Loss 8.3381e-01 (8.3661e-01)	Acc@1  78.52 ( 78.84)	Acc@5  94.14 ( 93.93)
Test: [ 40/196]	Time  1.925 ( 0.960)	Loss 8.9920e-01 (8.9030e-01)	Acc@1  76.17 ( 76.82)	Acc@5  94.92 ( 93.70)
Test: [ 50/196]	Time  1.005 ( 1.044)	Loss 4.8227e-01 (8.8155e-01)	Acc@1  86.33 ( 76.91)	Acc@5  97.27 ( 93.96)
Test: [ 60/196]	Time  0.540 ( 1.036)	Loss 1.2319e+00 (8.9501e-01)	Acc@1  69.14 ( 76.35)	Acc@5  89.84 ( 93.97)
Test: [ 70/196]	Time  0.906 ( 1.048)	Loss 1.0618e+00 (8.8639e-01)	Acc@1  72.66 ( 76.71)	Acc@5  90.62 ( 93.96)
Test: [ 80/196]	Time  0.786 ( 1.040)	L

tensor(70.3560, device='cuda:0')

In [17]:

#layer 2
model = models.vgg16(pretrained=True)
model = model.cuda()
    

weight = model.features[2].weight
weight_half = weight[:64,:64]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec in python'.format(total))

model.features[2].weight.data[:64,:64] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion)  

layer half weights size is:  torch.Size([64, 64, 3, 3])
need 0.004971 sec in python
Test: [  0/196]	Time  2.781 ( 2.781)	Loss 9.8434e+00 (9.8434e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.00 (  0.00)
Test: [ 10/196]	Time  0.282 ( 0.872)	Loss 9.5547e+00 (1.0004e+01)	Acc@1   0.00 (  0.07)	Acc@5   0.00 (  0.14)
Test: [ 20/196]	Time  2.747 ( 1.087)	Loss 1.0233e+01 (9.7589e+00)	Acc@1   0.00 (  0.06)	Acc@5   0.00 (  0.26)
Test: [ 30/196]	Time  0.269 ( 1.133)	Loss 9.7170e+00 (9.8220e+00)	Acc@1   0.00 (  0.04)	Acc@5   0.00 (  0.20)
Test: [ 40/196]	Time  3.225 ( 1.171)	Loss 1.0688e+01 (1.0043e+01)	Acc@1   0.00 (  0.03)	Acc@5   0.00 (  0.15)
Test: [ 50/196]	Time  0.368 ( 1.159)	Loss 8.6379e+00 (1.0060e+01)	Acc@1   0.00 (  0.02)	Acc@5   0.00 (  0.12)
Test: [ 60/196]	Time  1.438 ( 1.180)	Loss 7.7788e+00 (9.9459e+00)	Acc@1   0.00 (  0.02)	Acc@5   0.00 (  0.10)
Test: [ 70/196]	Time  0.268 ( 1.214)	Loss 9.7044e+00 (9.9197e+00)	Acc@1   0.00 (  0.02)	Acc@5   0.00 (  0.10)
Test: [ 80/196]	Time  2.725 ( 1.211)

tensor(0.2060, device='cuda:0')

In [18]:

#layer 3
model = models.vgg16(pretrained=True)
model = model.cuda()
    

weight = model.features[5].weight
weight_half = weight[:64,:64]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec in python'.format(total))

model.features[5].weight.data[:64,:64] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion) 

layer half weights size is:  torch.Size([64, 64, 3, 3])
need 0.004946 sec in python
Test: [  0/196]	Time  4.327 ( 4.327)	Loss 2.7252e+00 (2.7252e+00)	Acc@1  47.66 ( 47.66)	Acc@5  66.02 ( 66.02)
Test: [ 10/196]	Time  0.269 ( 1.082)	Loss 2.2212e+00 (2.9396e+00)	Acc@1  44.92 ( 38.07)	Acc@5  75.00 ( 62.71)
Test: [ 20/196]	Time  2.278 ( 1.020)	Loss 4.0896e+00 (2.6570e+00)	Acc@1  23.05 ( 42.54)	Acc@5  42.19 ( 67.32)
Test: [ 30/196]	Time  0.269 ( 0.947)	Loss 1.4247e+00 (2.7020e+00)	Acc@1  63.67 ( 42.54)	Acc@5  87.50 ( 66.44)
Test: [ 40/196]	Time  2.137 ( 0.961)	Loss 2.9775e+00 (2.7229e+00)	Acc@1  40.23 ( 41.30)	Acc@5  63.67 ( 66.31)
Test: [ 50/196]	Time  0.269 ( 0.915)	Loss 1.6722e+00 (2.6890e+00)	Acc@1  64.84 ( 41.84)	Acc@5  83.20 ( 67.19)
Test: [ 60/196]	Time  2.304 ( 0.937)	Loss 2.7069e+00 (2.7076e+00)	Acc@1  33.59 ( 41.28)	Acc@5  69.14 ( 66.95)
Test: [ 70/196]	Time  0.279 ( 0.917)	Loss 3.0455e+00 (2.6949e+00)	Acc@1  37.11 ( 41.84)	Acc@5  61.33 ( 67.16)
Test: [ 80/196]	Time  2.416 ( 0.944)

tensor(40.8920, device='cuda:0')

In [19]:
#layer 4
model = models.vgg16(pretrained=True)
model = model.cuda()
    

weight = model.features[7].weight
weight_half = weight[:128,:128]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec in python'.format(total))

model.features[7].weight.data[:128,:128] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion) 

layer half weights size is:  torch.Size([128, 128, 3, 3])
need 0.019129 sec in python
Test: [  0/196]	Time  4.085 ( 4.085)	Loss 7.9026e+00 (7.9026e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.00 (  0.00)
Test: [ 10/196]	Time  0.268 ( 1.082)	Loss 8.9828e+00 (9.0160e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.00 (  0.00)
Test: [ 20/196]	Time  0.268 ( 0.967)	Loss 8.2030e+00 (8.5999e+00)	Acc@1   0.00 (  0.02)	Acc@5   0.00 (  0.17)
Test: [ 30/196]	Time  2.633 ( 1.033)	Loss 7.4428e+00 (8.5567e+00)	Acc@1   0.00 (  0.03)	Acc@5   0.00 (  0.15)
Test: [ 40/196]	Time  0.267 ( 0.992)	Loss 8.5604e+00 (8.4831e+00)	Acc@1   0.00 (  0.02)	Acc@5   0.00 (  0.11)
Test: [ 50/196]	Time  3.459 ( 1.015)	Loss 8.0516e+00 (8.4540e+00)	Acc@1   0.00 (  0.02)	Acc@5   0.00 (  0.09)
Test: [ 60/196]	Time  0.268 ( 0.984)	Loss 9.0690e+00 (8.4374e+00)	Acc@1   0.00 (  0.01)	Acc@5   0.00 (  0.08)
Test: [ 70/196]	Time  2.949 ( 0.987)	Loss 7.3611e+00 (8.4463e+00)	Acc@1   0.00 (  0.01)	Acc@5   0.00 (  0.07)
Test: [ 80/196]	Time  0.274 ( 0.97

tensor(0.1180, device='cuda:0')

In [20]:
#layer 5
model = models.vgg16(pretrained=True)
model = model.cuda()
    

weight = model.features[10].weight
weight_half = weight[:128,:128]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec in python'.format(total))

model.features[10].weight.data[:128,:128] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion) 

layer half weights size is:  torch.Size([128, 128, 3, 3])
need 0.019444 sec in python
Test: [  0/196]	Time  4.318 ( 4.318)	Loss 1.6500e+00 (1.6500e+00)	Acc@1  57.03 ( 57.03)	Acc@5  85.55 ( 85.55)
Test: [ 10/196]	Time  0.269 ( 1.069)	Loss 2.3517e+00 (3.0739e+00)	Acc@1  41.80 ( 34.48)	Acc@5  74.61 ( 59.69)
Test: [ 20/196]	Time  2.442 ( 0.964)	Loss 3.5819e+00 (2.8832e+00)	Acc@1  31.64 ( 38.36)	Acc@5  48.44 ( 63.13)
Test: [ 30/196]	Time  0.269 ( 0.870)	Loss 1.4662e+00 (2.9710e+00)	Acc@1  64.06 ( 38.85)	Acc@5  89.06 ( 62.01)
Test: [ 40/196]	Time  1.915 ( 0.855)	Loss 2.9184e+00 (2.8452e+00)	Acc@1  32.81 ( 39.41)	Acc@5  64.84 ( 64.26)
Test: [ 50/196]	Time  0.268 ( 0.824)	Loss 1.4824e+00 (2.7029e+00)	Acc@1  63.67 ( 41.34)	Acc@5  85.55 ( 66.93)
Test: [ 60/196]	Time  2.485 ( 0.837)	Loss 4.1404e+00 (2.7521e+00)	Acc@1  19.14 ( 40.69)	Acc@5  40.23 ( 66.25)
Test: [ 70/196]	Time  0.270 ( 0.812)	Loss 2.5396e+00 (2.7660e+00)	Acc@1  46.88 ( 41.00)	Acc@5  67.58 ( 65.98)
Test: [ 80/196]	Time  1.894 ( 0.81

tensor(35.6940, device='cuda:0')

In [ ]:
#layer 6
model = models.vgg16(pretrained=True)
model = model.cuda()
    

weight = model.features[12].weight
weight_half = weight[:256,:256]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec in python'.format(total))

model.features[12].weight.data[:256,:256] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion) 

layer half weights size is:  torch.Size([256, 256, 3, 3])
need 0.087435 sec in python
Test: [  0/196]	Time  4.429 ( 4.429)	Loss 8.1701e+00 (8.1701e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.00 (  0.00)
Test: [ 10/196]	Time  0.268 ( 1.088)	Loss 8.0704e+00 (8.5797e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.00 (  0.00)
Test: [ 20/196]	Time  2.498 ( 1.013)	Loss 8.4822e+00 (8.3706e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.00 (  0.00)
Test: [ 30/196]	Time  0.269 ( 0.912)	Loss 7.9334e+00 (8.4736e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.00 (  0.00)
Test: [ 40/196]	Time  2.257 ( 0.914)	Loss 8.4983e+00 (8.4014e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.00 (  0.00)
Test: [ 50/196]	Time  0.326 ( 0.869)	Loss 7.3078e+00 (8.3408e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.00 (  0.00)


In [ ]:
#layer 7
model = models.vgg16(pretrained=True)
model = model.cuda()
    

weight = model.features[14].weight
weight_half = weight[:256,:256]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec in python'.format(total))

model.features[14].weight.data[:256,:256] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion) 

In [ ]:
#layer 8
model = models.vgg16(pretrained=True)
model = model.cuda()
    

weight = model.features[17].weight
weight_half = weight[:256,:256]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec in python'.format(total))

model.features[17].weight.data[:256,:256] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion) 

In [ ]:
#layer 9
model = models.vgg16(pretrained=True)
model = model.cuda()
    

weight = model.features[19].weight
weight_half = weight[:512,:512]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec in python'.format(total))

model.features[19].weight.data[:512,:512] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion) 

In [ ]:
#layer 10
model = models.vgg16(pretrained=True)
model = model.cuda()
    

weight = model.features[21].weight
weight_half = weight[:512,:512]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec in python'.format(total))

model.features[21].weight.data[:512,:512] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion)

In [ ]:
#layer 11
model = models.vgg16(pretrained=True)
model = model.cuda()
    

weight = model.features[24].weight
weight_half = weight[:512,:512]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec in python'.format(total))

model.features[24].weight.data[:512,:512] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion)

In [ ]:
#layer 12
model = models.vgg16(pretrained=True)
model = model.cuda()
    

weight = model.features[26].weight
weight_half = weight[:512,:512]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec in python'.format(total))

model.features[26].weight.data[:512,:512] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion)

In [ ]:
#laer 13
model = models.vgg16(pretrained=True)
model = model.cuda()
    

weight = model.features[28].weight
weight_half = weight[:512,:512]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec in python'.format(total))

model.features[28].weight.data[:512,:512] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion)